In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%run ../load_magic/storage.py
s = Storage()
[f's.{fn}' for fn in dir(s) if not fn.startswith('_')]

['s.attempt_to_pickle', 's.csv_exists', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.pickle_exists', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']

In [16]:

# Calculate the distribution of the letters in the garbled text
from collections import Counter
import re

alphabet_regex = re.compile('[^A-Za-z]+')
encoded_text = 'qrfcvgr zl vavgvny vzchyfr gung V fubhyq pbagenqvpg jubrire fnvq gung, "qngr n qhqr" jba bhg sbe zr'
garbled_words_list = alphabet_regex.split(encoded_text)
garbled_word_length_set = set([n for n in map(lambda x: len(x), garbled_words_list)])
letter_regex = re.compile('[A-Za-z]')
garbled_letters_list = letter_regex.findall(encoded_text)
garbled_letters_counter_obj = Counter(garbled_letters_list)
print([f'garbled_letters_counter_obj.{fn}' for fn in dir(garbled_letters_counter_obj) if not fn.startswith('_')])
garbled_letters_counter_obj

['garbled_letters_counter_obj.clear', 'garbled_letters_counter_obj.copy', 'garbled_letters_counter_obj.elements', 'garbled_letters_counter_obj.fromkeys', 'garbled_letters_counter_obj.get', 'garbled_letters_counter_obj.items', 'garbled_letters_counter_obj.keys', 'garbled_letters_counter_obj.most_common', 'garbled_letters_counter_obj.pop', 'garbled_letters_counter_obj.popitem', 'garbled_letters_counter_obj.setdefault', 'garbled_letters_counter_obj.subtract', 'garbled_letters_counter_obj.update', 'garbled_letters_counter_obj.values']


Counter({'g': 10, 'r': 8, 'q': 7, 'v': 7, 'n': 7, 'b': 6, 'f': 4, 'h': 4, 'u': 4, 'z': 3, 'a': 3, 'y': 3, 'e': 3, 'c': 2, 'p': 2, 'j': 2, 'l': 1, 'V': 1, 'i': 1, 's': 1})

In [47]:

# Store a list of valid words (in a dictionary) and a "normal" letter distibution for your language (in a list)
import time

GLOVE_840B_300D_PATH = '../data/300d/glove.840B.300d.txt'
GLOVE_6B_50D_PATH = '../data/50d/glove.6B.50d.txt'
t0 = time.time()
if s.pickle_exists('GLOVE_SMALL_DICT'):
    glove_dict = s.load_object('GLOVE_SMALL_DICT')
    counter_obj = s.load_object('GLOVE_SMALL_COUNTER')
else:
    glove_dict = {}
    letters_list = []
    with open(GLOVE_6B_50D_PATH, 'r', encoding='utf8') as infile:
        for line in infile:
            parts_list = line.split()
            word_str = parts_list[0]
            if not alphabet_regex.search(word_str):
                letter_count = len(word_str)
                if letter_count in garbled_word_length_set:
                    words_list = glove_dict.get(letter_count, [])
                    words_list.append(word_str)
                    glove_dict[letter_count] = words_list
                    for letter in word_str:
                        letters_list.append(letter)
    counter_obj = Counter(letters_list)
    s.store_objects(GLOVE_SMALL_DICT=glove_dict, GLOVE_SMALL_COUNTER=counter_obj)
t1 = time.time()
print(t1-t0, time.ctime(t1))
count_min = min(counter_obj.values())
letter_freq_dict = {k: v/count_min for k, v in counter_obj.items()}
letter_freq_dict

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\Wordle\saves\pkl\GLOVE_SMALL_DICT.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\Wordle\saves\pkl\GLOVE_SMALL_COUNTER.pkl
1.4233295917510986 Wed Mar 30 07:11:08 2022


{'t': 22.62368310155921, 'h': 14.742098609355246, 'e': 42.746734091866834, 'o': 28.435313948588284, 'f': 6.04677623261694, 'a': 49.959544879898864, 'n': 30.769490096923725, 'd': 15.132743362831858, 'i': 34.02697008006743, 'r': 31.0779603876949, 's': 29.11757269279393, 'w': 5.277286135693215, 'b': 11.003371260008429, 'y': 7.713442899283607, 'm': 15.335861778339654, 'v': 5.6131479140328695, 'u': 16.396544458491363, 'l': 23.79983143699958, 'p': 10.11504424778761, 'c': 15.321112515802781, 'g': 11.988621997471554, 'k': 10.138221660345554, 'j': 2.8158449220396125, 'x': 1.4517488411293722, 'q': 1.0, 'z': 3.723135271807838}

In [48]:

# Repeat: Set an array (rank) from all 26 letters to float (rank('A')=rank('B')=...=rank('Z')=0.0)
count_min = min(counter_obj.values())
lower_letter_freq_dict = {k: v/count_min for k, v in counter_obj.items() if k.lower() == k}
count_min = min(lower_letter_freq_dict.values())
lower_letter_freq_dict = {k: v/count_min for k, v in lower_letter_freq_dict.items()}
letter_ranked_tuples_list = sorted([(k, v) for k, v in lower_letter_freq_dict.items()], key=lambda x:x[1], reverse=True)
letter_ranked_tuples_list

[('a', 49.959544879898864), ('e', 42.746734091866834), ('i', 34.02697008006743), ('r', 31.0779603876949), ('n', 30.769490096923725), ('s', 29.11757269279393), ('o', 28.435313948588284), ('l', 23.79983143699958), ('t', 22.62368310155921), ('u', 16.396544458491363), ('m', 15.335861778339654), ('c', 15.321112515802781), ('d', 15.132743362831858), ('h', 14.742098609355246), ('g', 11.988621997471554), ('b', 11.003371260008429), ('k', 10.138221660345554), ('p', 10.11504424778761), ('y', 7.713442899283607), ('f', 6.04677623261694), ('v', 5.6131479140328695), ('w', 5.277286135693215), ('z', 3.723135271807838), ('j', 2.8158449220396125), ('x', 1.4517488411293722), ('q', 1.0)]

In [33]:

# Repeat: Set an array (rank) from all 26 letters to float (rank('A')=rank('B')=...=rank('Z')=0.0)
count_min = min(garbled_letters_counter_obj.values())
garbled_lower_letter_freq_dict = {k: v/count_min for k, v in garbled_letters_counter_obj.items() if k.lower() == k}
count_min = min(garbled_lower_letter_freq_dict.values())
garbled_lower_letter_freq_dict = {k: v/count_min for k, v in garbled_lower_letter_freq_dict.items()}
garbled_letter_ranked_tuples_list = sorted([(k, v) for k, v in garbled_lower_letter_freq_dict.items()], key=lambda x:x[1], reverse=True)
garbled_letter_ranked_tuples_list

[('g', 10.0), ('r', 8.0), ('q', 7.0), ('v', 7.0), ('n', 7.0), ('b', 6.0), ('f', 4.0), ('h', 4.0), ('u', 4.0), ('z', 3.0), ('a', 3.0), ('y', 3.0), ('e', 3.0), ('c', 2.0), ('p', 2.0), ('j', 2.0), ('l', 1.0), ('i', 1.0), ('s', 1.0)]

In [54]:

replacement_dict = {'v': 'i', 'V': 'I', 'z': 'w'}
gts_list = [garbled_tuple for garbled_tuple in garbled_letter_ranked_tuples_list if garbled_tuple[0] not in replacement_dict.keys()]
lts_list = [letter_tuple for letter_tuple in letter_ranked_tuples_list if letter_tuple[0] not in replacement_dict.values()]
for garbled_tuple, letter_tuple in zip(gts_list, lts_list):
    replacement_dict[garbled_tuple[0]] = letter_tuple[0]
print(encoded_text)
decoded_text = ''.join(map(lambda x: replacement_dict.get(x, x), encoded_text))
print(decoded_text)
print(decoded_text[::-1])

qrfcvgr zl vavgvny vzchyfr gung V fubhyq pbagenqvpg jubrire fnvq gung, "qngr n qhqr" jba bhg sbe zr
reodiae wb iuiainm iwdlmoe atna I otslmr hsuacnriha gtsekec onir atna, "rnae n rlre" gsu sla psc we
ew csp als usg "erlr n eanr" ,anta rino cekestg ahirncaush rmlsto I anta eomldwi mniaiui bw eaidoer


In [51]:

glove_dict[2]

['of', 'to', 'in', 'on', 'is', 'he', 'as', 'it', 'by', 'at', 'an', 'be', 'or', 'we', 'up', 'if', 'no', 'do', 'us', 'so', 'my', 'go', 'me', 'al', 'de', 'eu', 'un', 'tv', 'ii', 'ca', 'am', 'la', 'wo', 'el', 'ap', 'km', 'co', 're', 'bc', 'uk', 'ex', 'st', 'li', 'le', 'gm', 'ad', 'mm', 'ah', 'en', 'oh', 'pm', 'cd', 'fc', 'ma', 'hu', 'ed', 'ee', 'wu', 'da', 'uh', 'sa', 'di', 'ok', 'du', 'fm', 'et', 'pc', 'hk', 'bp', 'ho', 'nt', 'au', 'fa', 'iv', 'nn', 'ag', 'il', 'na', 'mp', 'mr', 'dr', 'ac', 'ld', 'ny', 'kg', 'ni', 'vs', 'yu', 'ep', 'se', 'dj', 'bt', 'cm', 'dc', 'ai', 'lu', 'pa', 'hp', 'ha', 'mi', 'ss', 'vi', 'su', 'ab', 'es', 'ge', 'ki', 'ba', 'ft', 'xi', 'gp', 'id', 'lo', 'jo', 'yi', 'ji', 'fu', 'xu', 'td', 'lp', 'ur', 'fe', 'te', 'ih', 'mo', 'hi', 'bo', 'mg', 'nl', 'sr', 'nz', 'os', 'ga', 'ms', 'eh', 'ce', 'hd', 'pg', 'er', 'jp', 'va', 'sc', 'oz', 'md', 'oo', 'ty', 'fi', 'cc', 'ul', 'si', 'rn', 'aa', 'ip', 'tb', 'sm', 'op', 'ng', 'pt', 'po', 'ya', 'th', 'sk', 'uc', 'ka', 'im', 'ec', 'gb

In [ ]:

# Compare your garbled distribution with the normal one and regarble your text according to that.

# Repeat: Set an array (rank) from all 26 letters to float (rank('A')=rank('B')=...=rank('Z')=0.0)

# Check the words in the produced text against words in the dictionary.
# If a word is in the dictionary, raise the rank of that word's letters (something like: add a standard value, say 1.0).
# In other words calculate Score (a function of total rank and number of words in dictionary).

# Save text into High score table (if score high enough).

# If all words are in the dictionary or if the total rank is high enough or if the loop was done more than 10000 times, End.

# If not, choose randomly two letters and interchange them.
# But with a deviated distribution, letters with high rank should have less chances of being interchanged.

# Repeat.

# End: Print High score texts.